In [214]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from tqdm import tqdm

## Load motions from calico tabs

In [273]:
urls = """
https://hwsrr2025.calicotab.com/_/
https://usudc2025.calicotab.com/_/
https://wudc2025.calicotab.com
https://paudc2024.calicotab.com/paudcK24/
https://edudrift.calicotab.com/abp2024/
https://neadc2024.calicotab.com/neadc2024/
https://eudc2024.calicotab.com/_/
https://naudc2024.calicotab.com/naudc2024/ 
https://wudc2024.calicotab.com/wudc/
https://edudrift.calicotab.com/neadc2023/
https://usudc2023.calicotab.com/usudc2023/
https://edudrift.calicotab.com/neadc2023/tab/team/  
https://midp.calicotab.com/mabp2023/
https://eudc2023.calicotab.com/_/
https://naudc2023.calicotab.com/
https://hwsrr2023.calicotab.com/hwsrr2023/
https://wudc2023.calicotab.com/wudc/
https://paudc2022.calicotab.com/PAUDC2022/ 
https://edudriftbackup.calicotab.com/abp2022/
https://usudc2022.calicotab.com/usudc2022/
http://neadc2022.calicotab.com/2022/
https://eudc2022.calicotab.com/eudc2022/
http://wudc2022.calicotab.com/wudc/
https://naudc2021.calicotab.com/_/tab/team/
http://neadc.calicotab.com/neadc/
https://awdc2021.herokuapp.com/sydneyAWDC/
https://abp2021.calicotab.com/abp2021/
https://mace.calicotab.com/sanudc2021/
https://mace.calicotab.com/paudc2021/
https://eudc2021.calicotab.com/eudc2021/
https://wudckorea.calicotab.com/2021/
https://naudc2020.herokuapp.com/naudc2020/s
https://rrtabs.herokuapp.com/paudc2020/
https://awdc2020.herokuapp.com/awdc/results/round/11/
https://abp2020.calicotab.com/abp2020/
https://eudc2020.calicotab.com/eudc2020/
https://wudc2020.calicotab.com/wudc2020/
https://usudc2020.calicotab.com/usudc2020/
https://wudc2018.calicotab.com/wudc2018/
https://wudc2014.calicotab.com/wudc2014/
https://docs.google.com/spreadsheets/d/1ryOKiMPPW7tP-lsvkR8xM6u1r4JmrSkj4UMitseckKM/pub?gid=0#
https://edudrift.calicotab.com/uadc2025/
https://uadc2024.au-tab.app/uadc2024/
http://malaysiauadc2022.calicotab.com/malaysiauadc2022/ 
https://bduadc2020.calicotab.com/uadc2020/
https://edudrift.calicotab.com/australs2025/
https://edudrift.calicotab.com/australs2024/
https://krabiaustrals2023.calicotab.com/australs2023/
https://australs2022.calicotab.com/australs2022/
https://australs2021.calicotab.com/
https://australs2020.calicotab.com/australs2020/
http://easters2025.calicotab.com/easters2025/
http://easters2024.calicotab.com/easters2024/?fbclid=IwAR0sUz8N7MtT4z_oaK1AT0ADfBs6xaN0pI8OKc7UcIcEyIzl9q3KfI8czRc_aem_AXCb6j-ifosiNs7_5os8Toueas-lu4DEPfiNtzMYdm97tPYCG9X3Z_hVbLv9gxf6fv9cCQf6odNf6HRdEJvCfJhY
http://easters2023.calicotab.com/easters2023/motions/
https://easters2022.calicotab.com/easters2022/
https://onlineeasters2021.calicotab.com/onlineeasters2021/
https://wsdc2024.calicotab.com/wsdc2024/
https://docs.google.com/spreadsheets/u/0/d/1lralzhC-MUcXZsEOQ4i6b_Ep0nLzbU4b_kd1aIxgb1M/htmlview#
http://tabbycat.idebate.net/
https://docs.google.com/spreadsheets/d/1oBOCHbLLJAtekaVPP0tZldolpZjFY5-x1eoU-CDGRJU/edit?fbclid=IwAR0m9Y3hFxq0N7HUrQVws8htQRMvSLN-3iyP-GT0vy-brYZJr8mJsDSxXoI#gid=892844460
https://tinyurl.com/tab
"""

# https://cmude2024.calicotab.com/open/
# https://cmude2023.calicotab.com/cmude2023/
# https://cmude2022.calicotab.com/cmude2022/
# https://cmude2021.calicotab.com/cmudeecuador2021/
# https://cmudemadrid2020.calicotab.com/cmude2020/
# https://cmude2019.calicotab.com/cmude2019/
# https://cmude2018.calicotab.com/

In [285]:
calico_tabs = []

for url in urls.split('\n'): 
    if 'calicotab' not in url:
        continue

    pattern = re.compile(r'https?:\/\/(\w+)\.calicotab\.com\/(\w+)\/')
    match = re.match(pattern, url)

    if match is None:
        print(f'[WARN] Could not parse url: {url}')
        continue

    subdomain = match.group(1)
    slug = match.group(2)

    calico_tabs.append((subdomain, slug))

    # print(subdomain, slug)

[WARN] Could not parse url: https://wudc2025.calicotab.com
[WARN] Could not parse url: https://naudc2023.calicotab.com/
[WARN] Could not parse url: https://australs2021.calicotab.com/


In [286]:
calico_tabs[:5]

[('hwsrr2025', '_'),
 ('usudc2025', '_'),
 ('paudc2024', 'paudcK24'),
 ('edudrift', 'abp2024'),
 ('neadc2024', 'neadc2024')]

In [303]:
rows = []

for subdomain, slug in calico_tabs: 
    api_url = f"https://{subdomain}.calicotab.com/api/v1/tournaments/{slug}/motions"

    response = requests.get(api_url) 
    try: 
        assert response.status_code == 200, f"Failed to fetch {api_url}: {response.status_code}"
    except: 
        print(f'[WARN] Failed to fetch API for url: {api_url}')
        continue

    motions = response.json()
    for m in motions:

        if len(m['rounds']) <= 0: #skip unused (test) motions
            continue

        row = {
            'Motion': m['text'],
            'Info Slide': m['info_slide_plain'],
            'Tournament': subdomain + ':' + slug,
        }

        rows.append(row)


df = pd.DataFrame(rows)
df.tail()

,Motion,Info Slide,Tournament
0,This house prefers that progressive journalist...,,hwsrr2025:_
1,This house regrets the Indian Premier League.,The Indian Premier League (IPL) is a domestic ...,hwsrr2025:_
2,This house believes that breaking up large mea...,,hwsrr2025:_
3,This house prefers a world where genius only d...,,hwsrr2025:_
4,This house believes that East African national...,,hwsrr2025:_


## Scraping from Tabbycat 

DEPRECATED!! Only works for WSDC. 

In [259]:
# calico_tabs = [
#     ('nsdc25', 'NordicSDC2025'),
#     ('eurosdc25', 'eurosdc-25'),
#     # ('esdc', 'esdc2025'),
#     ('argument', 'eco2025'),
#     ('argument', 'ECO2024'),
#     ('nordics2023', 'nordics2023')
# ]

In [260]:
def parse_motion(div): 
    """
    Given a motion top-level div, parse out the relevant details.
    """
    round = div.find('span', class_="badge").get_text(strip=True)
    motion = div.find('h4').find(string=True, recursive=False).strip()

    info_slide = div.find(string=re.compile("View Info Slide"))
    info_slide_text = None

    if info_slide is not None:
        info_slide_text = info_slide.find_next('div', class_='modal-body').get_text(strip=True)

    # regex match the wins for prop and opp
    text = div.get_text()

    re_prop = re.compile(r"(\d+) (Prop|Gov) win")
    re_opp = re.compile(r"(\d+) Opp win")

    prop_wins = re.findall(re_prop, text)[0][0]
    opp_wins = re.findall(re_opp, text)[0][0]

    balance = div.find(string=re.compile('balance')).get_text(strip=True)

    return motion, round, info_slide_text, prop_wins, opp_wins, balance

In [ ]:
data = {
    "Tournament": [],
    "Motion": [],
    "Round": [],
    "Info Slide": [],

    "Prop wins": [],
    "Opp wins": [],
    "Balance": [],
}

for subdomain, slug in calico_tabs:
    calico_url = f"https://{subdomain}.calicotab.com/{slug}"

    motions_suburl = f"motions/statistics"
    motions_url = f"{calico_url}/{motions_suburl}"

    response = requests.get(motions_url) 
    try: 
        assert response.status_code == 200, f"Failed to fetch {motions_url}: {response.status_code}"
    except: 
        continue

    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

    try:
        motion_divs = soup.find_all('div', class_="list-group mt-3")
        print(f"Found {len(motion_divs)} motions from {slug}.")

        for start in tqdm(range(len(motion_divs))):
            div = motion_divs[start]
            motion, round, info_slide_text, prop_wins, opp_wins, balance = parse_motion(div)
            

            data["Motion"].append(motion)
            data["Round"].append(round)
            data["Info Slide"].append(info_slide_text)
            data["Prop wins"].append(int(prop_wins))
            data["Opp wins"].append(int(opp_wins))
            data["Balance"].append(balance)
            data["Tournament"].append(slug)
    except: 
        print(f'[WARN] could not parse motions for url: {calico_url}')

Found 11 motions from _.


  0%|          | 0/11 [00:00<?, ?it/s]

[WARN] could not parse motions for url: https://hwsrr2025.calicotab.com/_


Found 10 motions from _.


  0%|          | 0/10 [00:00<?, ?it/s]

...

[WARN] could not parse motions for url: https://naudc2024.calicotab.com/naudc2024


Found 19 motions from wudc.


  0%|          | 0/19 [00:00<?, ?it/s]

[WARN] could not parse motions for url: https://wudc2024.calicotab.com/wudc


KeyboardInterrupt: 

In [218]:
df1 = pd.DataFrame(data)
df1.tail()

,Tournament,Motion,Round,Info Slide,Prop wins,Opp wins,Balance
45,nordics2023,THBT the European Union should abolish the req...,Round 5,On issues considered sensitive to the EU (incl...,6,7,probably balanced
46,nordics2023,THR the expectation that committed romantic re...,Round 6,"Just like a lot of you, we were very tired and...",10,3,imbalanced at 10% level
47,nordics2023,THR the aesthetisation of suffering,Quarterfinals,Aesthetisation is a depiction or glorification...,4,0,balance inconclusive
48,nordics2023,THBT criminal justice policy should be decided...,Semifinals,Technocrats are decision-makers who are select...,1,1,balance inconclusive
49,nordics2023,THR religious faith being predominantly experi...,Grand Final,None,1,0,balance inconclusive


## Parse WSDC motions from .txt on GitHub
https://github.com/tokyodebate/motions/blob/main/International/WSDC.txt

In [219]:
import requests

url = 'https://raw.githubusercontent.com/tokyodebate/motions/refs/heads/main/International/WSDC.txt'

response = requests.get(url)
wsdc_motions_raw = response.text

In [220]:
def count_tabs(line: str):
    """
    Count the number of tabs in a line.
    """
    return line.count('\t')

In [221]:
data2 = {
    "Tournament": [],
    "Motion": [],
    "Round": [],
    "Info Slide": [],

    # "Prop wins": [],
    # "Opp wins": [],
    # "Balance": [],
}

def parse_wsdc_motions():
    lines = wsdc_motions_raw.split('\n')
    lines = [line.strip('\n') for line in lines]
    lines = lines[1:] # Skip the header

    tournament = None
    round = None

    for i in range(len(lines)):
        curr = lines[i]

        if count_tabs(curr) == 1: 
            tournament = curr.strip()
        elif count_tabs(curr) == 2:
            round = curr.strip()
        elif count_tabs(curr) == 3:
            next_line = lines[i+1] if i+1 < len(lines) else None
            info_text = next_line.strip() if next_line and count_tabs(next_line) == 4 else None

            motion = curr.strip()   
            data2["Motion"].append(motion)
            data2["Round"].append(round)
            data2["Info Slide"].append(info_text)
            data2["Tournament"].append(tournament)

        else:
            continue

parse_wsdc_motions()

In [222]:
df2 = pd.DataFrame(data2)
df2.value_counts('Tournament')

Tournament
Netherlands WSDC 2022                                                           21
WSDC Mexico 2020                                                                20
24th World Schools Debating Championships 2012, South Africa                    13
21st World Schools Debating Championships 2009 in Athens, Greece                13
Vietnam WSDC 2023                                                               13
30th World Schools Debating Championships 2018, Croatia and Slovenia            13
29th World Schools Debating Championships 2017, Indonesia                       13
28th World Schools Debating Championships 2016, Germany                         13
27th World Schools Debating Championships 2015, Singapore                       13
WSDC Thailand 2019                                                              13
16th World Schools Debating Championships 2004 in Stuttgart, Germany            13
15th World Schools Debating Championships 2003 in Lima, Peru                

In [223]:
df = pd.concat([df1, df2], ignore_index=True)
df.head()

,Tournament,Motion,Round,Info Slide,Prop wins,Opp wins,Balance
0,NordicSDC2025,This House Supports gentle parenting becoming ...,Round 1,Gentle parenting is a parenting style that say...,16.0,1.0,imbalanced at 50% level
1,NordicSDC2025,THR the creation of the series and film industry,Round 2,None,13.0,1.0,probably balanced
2,NordicSDC2025,This House Believes That it is in the interest...,Round 3,Democratic backsliding is a process of regime ...,15.0,1.0,probably balanced
3,NordicSDC2025,This House Opposes the Globalization of the Ma...,Round 4,“Major European Football Leagues” include the ...,10.0,1.0,imbalanced at 50% level
4,NordicSDC2025,This House Would implement a weekly 'blackout ...,Round 5,A 'blackout day' refers to a day where all soc...,13.0,1.0,probably balanced


## Data polishing 

In [304]:
df['Motion'] = df['Motion'].astype("string")

### Duplicates

In [305]:
# check for duplicates on motion and round 
# motions can be duplicated (if same is used in two different rounds)
duplicates = df[df.duplicated(subset=['Motion'], keep=False)]

if len(duplicates) > 0:
    raise ValueError(f"Found {len(duplicates)} duplicates in the data. Please check the data for inconsistencies.")

ValueError: Found 66 duplicates in the data. Please check the data for inconsistencies.

### Tournament year

In [307]:
def get_year(tournament: str):
    pattern = re.compile(r'\d{4}')
    match = pattern.search(tournament)

    if tournament == 'eurosdc-25':
        return 2025

    if match:
        return int(match.group(0))
    else:
        return None

df['Year'] = df['Tournament'].apply(get_year)
df.value_counts('Year')

Year
2022.0    207
2024.0    173
2020.0    170
2023.0    165
2025.0    127
2021.0    111
2018.0     26
2014.0     19
Name: count, dtype: int64

### Motion types

In [318]:
def fix_motions(motion: str):
    pattern = re.compile(r'(A|B|C)\W ') #australs bs that adds A, B, C motion types

    if match := re.match(pattern, motion): 
        motion = motion[match.span()[1]:]
    

    if motion == "This House as the environmental movement would support the use of extremist tactics":
        return "This House, as the environmental movement, would support the use of extremist tactics."
    
    if motion == "TH, as the average 25 year old Romanian, would wait for their 'statistical soulmate'(josh is sad and lonely)":
        return motion.replace('would', "W")

    if motion.startswith('That'):
        motion = 'This House believes that' + motion[4:]
    
    return motion

df['Motion'] = df['Motion'].apply(fix_motions)

In [319]:
def extract_motion_header(motion: str): 
    # https://regex101.com/r/XB1k1w/1
    pattern = re.compile(r"(?:(T|t)his (h|H)ouse?|TH)[ ,]?(?:,.*?, )?(?P<type>\w+)\s?")

    match = re.search(pattern, motion)
    return match

In [320]:
full_to_abbr = {
    'BT': "believes",
    'B': "believes",
    "S": "supports",
    "O": "opposes",
    "W": "would",
    "P": "prefers",
    "R": "regrets",
}


def get_motion_type(motion: str): 
    match = extract_motion_header(motion)

    if match: 
        motion_type = match.group('type')
        all_caps = all(map(str.isupper, motion_type))

        if all_caps:
            return f'{full_to_abbr[motion_type]}'
        else: 
            return f'{motion_type.lower()}'
    else:
        print(f"Could not determine motion type for: {motion}")

df["Motion Type"] = df["Motion"].apply(get_motion_type)

Could not determine motion type for: Assuming an initial agreement is possible, that it is the interest of DA to include EFF in its (DA's) coalition for the 2024 election. [coalition]
Could not determine motion type for:  That the US should attempt to aggressively contain China's capacity to build AI systems (e.g. by limiting China's ability to skill and retain computer scientists, attract US capital into Chinese AI companies, or access parts needed to build AI systems where the US has significant market share, such as semiconductors).
Could not determine motion type for: Assuming the technology exists, that it is immoral to create simulated high-fidelity alternate universes.
Could not determine motion type for:  That we support memory laws


## Vectorization
NB! This step requires access to a Google Cloud project with billing enabled. If desired, you can sign up for a $300 free trial that will be plenty to cover the costs of vectorization.

Alternatively, all vector space information is already included in the attached `motions.csv` file.

In [157]:
%pip install --upgrade -q google-genai vertexai dotenv


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [158]:
# only run this once
# !gcloud auth application-default login

In [330]:
import vertexai
from vertexai.language_models import TextEmbeddingModel
from dotenv import load_dotenv
import os

load_dotenv()
vertexai.init(project=os.environ['VERTEXAI_PROJECT_ID'])

In [333]:
def clean_motion(motion: str):
    match = extract_motion_header(motion)

    if match is None:
        print(f'[WARN] No header found for motion: {motion}')
        return motion

    header_end = match.span()[1]
    chopped = motion[header_end:].lower()

    filler_words = ['that', 'the', 'it', 'is']

    flag = True
    while flag: 
        flag = False

        for w in filler_words: 
            if chopped.startswith(w): 
                chopped = chopped.replace(w, '')
                flag=True
    
    punctuations = ['.', ',' ';']
    for p in punctuations: 
        chopped = chopped.replace(p, '')
    return chopped


In [334]:
df['Motion (cleaned)'] = df['Motion'].map(clean_motion)
df.head()

[WARN] No header found for motion: Assuming an initial agreement is possible, that it is the interest of DA to include EFF in its (DA's) coalition for the 2024 election. [coalition]
[WARN] No header found for motion:  That the US should attempt to aggressively contain China's capacity to build AI systems (e.g. by limiting China's ability to skill and retain computer scientists, attract US capital into Chinese AI companies, or access parts needed to build AI systems where the US has significant market share, such as semiconductors).
[WARN] No header found for motion: Assuming the technology exists, that it is immoral to create simulated high-fidelity alternate universes.
[WARN] No header found for motion:  That we support memory laws


,Motion,Info Slide,Tournament,Year,Motion Type,Motion (cleaned)
0,This house prefers that progressive journalist...,,hwsrr2025:_,2025.0,prefers,progressive journalists focus their efforts o...
1,This house regrets the Indian Premier League.,The Indian Premier League (IPL) is a domestic ...,hwsrr2025:_,2025.0,regrets,indian premier league
2,This house believes that breaking up large mea...,,hwsrr2025:_,2025.0,believes,breaking up large meat companies is in the in...
3,This house prefers a world where genius only d...,,hwsrr2025:_,2025.0,prefers,a world where genius only develops after adult...
4,This house believes that East African national...,,hwsrr2025:_,2025.0,believes,east african national governments should prio...


In [ ]:
from typing import Optional

def embed(texts, output_dims: Optional[int] = None):
  model_id = 'text-embedding-005'
  # model_id = 'gemini-embedding-001'
  model = TextEmbeddingModel.from_pretrained(model_id)
  embeddings = model.get_embeddings(texts, output_dimensionality=output_dims)

  return embeddings

In [340]:
# vertexai allows a max of 250 input elements at once
chunk_size = 100

for start in tqdm(range(0, len(df), chunk_size)):
    end = start + chunk_size
    motion_strings = df['Motion (cleaned)'].values[start:end]

    embeddings = embed(list(motion_strings))
    df.loc[start:end-1, 'Embedding'] = embeddings #NB! pandas uses an inclusive end index (hence the end-1)

    # if start > 100:
    #     break

100%|██████████| 11/11 [00:35<00:00,  3.18s/it]


## Save data locally

In [342]:
#TODO: maybe better to store each value in its own column? or just use a diff file format specifically for floats
from vertexai.language_models import TextEmbedding

df['Embedding'] = df['Embedding'].map(lambda e: e.values if isinstance(e, TextEmbedding) else None) #get only the numbers

In [344]:
import os

data_dir = 'data'

os.makedirs(data_dir, exist_ok=True)
df.to_csv(os.path.join(data_dir, 'motions_bp.csv'))